<img src="images/arm_logo.png" width=500 alt="ARM Logo"></img>

# Bankhead National Forest - RadClss Example

## Overview

The Extracted Radar Columns and In-Situ Sensors (RadClss) Value-Added Product (VAP) is
a dataset containing in-situ ground observations matched to CSAPR-2 radar columns above ARM Mobile Facility (AMF-3) supplemental sites of interest. 

RadCLss is intended to provide a dataset for algorthim development and validation of precipitation retrievals.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| [GeoPandas](https://geopandas.org/en/stable/docs.html) | Necessary | Familiarity with Geospatial Plotting|
| [Py-ART / Radar Foundations](https://projectpythia.org/radar-cookbook/README.html) | Necessary | Basics of Weather Radar | 

- **Time to learn**: 30 minutes

## Current List of Supported Sites of Interest
| Site  | Lat   | Lon   |
| ----- | ----- | ----- |
| M1    | 34.34525 | -87.33842 |
| S4    | 34.46451 | -87.23598 |
| S20   | 34.65401 | -87.29264 |
| S30   | 34.38501 | -86.92757 |
| S40   | 34.17932 | -87.45349 |



<img src="images/bnf-in-situ-locations.png" width=1500 alt="BNF Sensors"></img>

## Current List of Supported In-Situ Ground Observations
- Surface Meteorological Instrumentation [MET] (DOI: 10.5439/1786358)
    - M1, S20, S30, S40
- Balloon-borne sounding system [SONDEWNPN] (DOI: 10.5439/1595321)*
    - M1 

## Planned List of Supported In-Situ Ground Observations
- *Pluvio Weighing Bucket Precipitation Gauge [WBPLUVIO2] (DOI: )*
- *Surface Meteorological Instrumentation [MET] (DOI: )*
- *Laser Disdrometer [LD] (DOI: )*

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)


import glob
import os
import datetime

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

from dask.distributed import Client, LocalCluster

import act
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## Define Processing Variables

In [13]:
# Define the desired processing date for the BNF CSAPR-2 in YYYY-MM-DD format.
DATE = "2025-01-09"
# Define the directory where the BNF CSAPR-2 CMAC files are located.
##RADAR_DIR = "/gpfs/wolf2/arm/atm124/proj-shared/bnf/bnfcsapr2cmacS3.c1/"
RADAR_DIR = "/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/"
# Define an output directory for downloaded ground instrumentation
##INSITU_DIR = '/gpfs/wolf2/arm/atm124/proj-shared/bnf/in_situ/'
INSITU_DIR = "/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/"

In [3]:
# Define ARM Username and ARM Token with ARM Live service for downloading ground instrumentation via ACT.DISCOVERY
# With your ARM username, you can find your ARM Live token here: https://adc.arm.gov/armlive/
##ARM_USERNAME = os.getenv("ARM_USERNAME")
##ARM_TOKEN = os.getenv("ARM_TOKEN")
ARM_USERNAME = "jrobrien"
ARM_TOKEN =  "5c339110fc936ee3"

## Define Functions 

In [4]:
def subset_points(nfile, **kwargs):
    """
    Subset a radar file for a set of latitudes and longitudes
    utilizing Py-ART's column-vertical-profile functionality.

    Parameters
    ----------
    file : str
        Path to the radar file to extract columns from
    nsonde : list
        List containing file paths to the desired sonde file to merge

    Calls
    -----
    radar_start_time
    merge_sonde

    Returns
    -------
    ds : xarray DataSet
        Xarray Dataset containing the radar column above a give set of locations
    
    """
    ds = None
    
    # Define the splash locations [lon,lat]
    M1 = [34.34525, -87.33842]
    S4 = [34.46451,	-87.23598]
    S20 = [34.65401, -87.29264]
    S30	= [34.38501, -86.92757]
    S40	= [34.17932, -87.45349]

    sites = ["M1", "S4", "S20", "S30", "S40"]

    # Zip these together!
    lats, lons = list(zip(M1,
                          S4,
                          S20,
                          S30,
                          S40))
    try:
        # Read in the file
        radar = pyart.io.read(nfile)
        # Check for single sweep scans
        if np.ma.is_masked(radar.sweep_start_ray_index["data"][1:]):
            radar.sweep_start_ray_index["data"] = np.ma.array([0])
            radar.sweep_end_ray_index["data"] = np.ma.array([radar.nrays])
    except:
        radar = None

    if radar:
        if radar.scan_type != "rhi":
            # Easier to map the nearest sonde file to radar gates before extraction
            if 'sonde' in kwargs:
                # variables to discard when reading in the sonde file
                exclude_sonde = ['base_time', 'time_offset', 'lat', 'lon', 'qc_pres',
                                 'qc_tdry', 'qc_dp', 'qc_wspd', 'qc_deg', 'qc_rh',
                                 'qc_u_wind', 'qc_v_wind', 'qc_asc']
        
                # find the nearest sonde file to the radar start time
                radar_start = datetime.datetime.strptime(nfile.split('/')[-1].split('.')[-3] + '.' + nfile.split('/')[-1].split('.')[-2], 
                                                         '%Y%m%d.%H%M%S'
                )
                sonde_start = [datetime.datetime.strptime(xfile.split('/')[-1].split('.')[2] + 
                                                          '-' + 
                                                          xfile.split('/')[-1].split('.')[3], 
                                                          '%Y%m%d-%H%M%S') for xfile in kwargs['sonde']
                          ]
                # difference in time between radar file and each sonde file
                start_diff = [radar_start - sonde for sonde in sonde_start]

                # merge the sonde file into the radar object
                ds_sonde = act.io.read_arm_netcdf(kwargs['sonde'][start_diff.index(min(start_diff))], 
                                                  cleanup_qc=True, 
                                                  drop_variables=exclude_sonde)
   
                # create list of variables within sonde dataset to add to the radar file
                for var in list(ds_sonde.keys()):
                    if var != "alt":
                        z_dict, sonde_dict = pyart.retrieve.map_profile_to_gates(ds_sonde.variables[var],
                                                                                 ds_sonde.variables['alt'],
                                                                                 radar)
                    # add the field to the radar file
                    radar.add_field_like('corrected_reflectivity', "sonde_" + var,  sonde_dict['data'], replace_existing=True)
                    radar.fields["sonde_" + var]["units"] = sonde_dict["units"]
                    radar.fields["sonde_" + var]["long_name"] = sonde_dict["long_name"]
                    radar.fields["sonde_" + var]["standard_name"] = sonde_dict["standard_name"]
                    radar.fields["sonde_" + var]["datastream"] = ds_sonde.datastream

                del radar_start, sonde_start, ds_sonde
                del z_dict, sonde_dict
        
            column_list = []
            for lat, lon in zip(lats, lons):
                # Make sure we are interpolating from the radar's location above sea level
                # NOTE: interpolating throughout Troposphere to match sonde to in the future
                try:
                    da = (
                        pyart.util.columnsect.column_vertical_profile(radar, lat, lon)
                        .interp(height=np.arange(radar.altitude['data'][0], 10050, 50))
                    )
                except ValueError:
                    da = pyart.util.columnsect.column_vertical_profile(radar, lat, lon)
                    
                # Add the latitude and longitude of the extracted column
                da["latitude"], da["longitude"] = lat, lon
                # Time is based off the start of the radar volume
                dt = pd.to_datetime(radar.time["data"], unit='ms')[-1]
                da["time"] = [dt]
                column_list.append(da)
        
            # Concatenate the extracted radar columns for this scan across all sites    
            ds = xr.concat([data for data in column_list if data], dim='station')
            ds["station"] = sites
            # Add attributes for Time, Latitude, Longitude, and Sites
            ds.time.attrs.update(long_name=('Time in Seconds that Cooresponds to the Start'
                                            + " of each Individual Radar Volume Scan before"
                                            + " Concatenation"),
                                 description=('Time in Seconds that Cooresponds to the Minimum'
                                              + ' Height Gate'))
            ds.station.attrs.update(long_name="Bankhead National Forest AMF-3 In-Situ Ground Observation Station Identifers")
            ds.latitude.attrs.update(long_name='Latitude of BNF AMF-3 Ground Observation Site',
                                     units='Degrees North')
            ds.longitude.attrs.update(long_name='Longitude of BNF AMF-3 Ground Observation Site',
                                     units='Degrees East')
            # delete the radar to free up memory
            del radar, column_list, da
        else:
            # delete the rhi file
            del radar
    return ds

In [5]:
def match_datasets_act(column, ground, site, discard, resample='sum', DataSet=False):
    """
    Time synchronization of a Ground Instrumentation Dataset to 
    a Radar Column for Specific Locations using the ARM ACT package
    
    Parameters
    ----------
    column : Xarray DataSet
        Xarray DataSet containing the extracted radar column above multiple locations.
        Dimensions should include Time, Height, Site
             
    ground : str; Xarray DataSet
        String containing the path of the ground instrumentation file that is desired
        to be included within the extracted radar column dataset. 
        If DataSet is set to True, ground is Xarray Dataset and will skip I/O. 
             
    site : str
        Location of the ground instrument. Should be included within the filename. 
        
    discard : list
        List containing the desired input ground instrumentation variables to be 
        removed from the xarray DataSet. 
    
    resample : str
        Mathematical operational for resampling ground instrumentation to the radar time.
        Default is to sum the data across the resampling period. Checks for 'mean' or 
        to 'skip' altogether. 
    
    DataSet : boolean
        Boolean flag to determine if ground input is an Xarray Dataset.
        Set to True if ground input is Xarray DataSet. 
             
    Returns
    -------
    ds : Xarray DataSet
        Xarray Dataset containing the time-synced in-situ ground observations with
        the inputed radar column 
    """
    # Check to see if input is xarray DataSet or a file path
    if DataSet == True:
        grd_ds = ground
    else:
        # Read in the file using ACT
        grd_ds = act.io.read_arm_netcdf(ground, cleanup_qc=True, drop_variables=discard)
        # Default are Lazy Arrays; convert for matching with column
        grd_ds = grd_ds.compute()
        # Check to see if file is the RWP, 
        if 'rwp' in ground[0].split('/')[-1]:
            # adjust the RWP heights above ground level
            grd_ds['height'] = grd_ds.height.data + grd_ds.alt.data
        if 'ceil' in ground[0].split('/')[-1]:
            # correct ceilometer backscatter 
            grd_ds = act.corrections.correct_ceil(grd_ds, var_name='backscatter')
            # Rename the range dimension and apply altitude 
            grd_ds = grd_ds.rename({'range' : 'height'})
            grd_ds['height'] = grd_ds.height.data + grd_ds.alt.data
        
    # Remove Base_Time before Resampling Data since you can't force 1 datapoint to 5 min sum
    if 'base_time' in grd_ds.data_vars:
        del grd_ds['base_time']
        
    # Check to see if height is a dimension within the ground instrumentation. 
    # If so, first interpolate heights to match radar, before interpolating time.
    if 'height' in grd_ds.dims:
        grd_ds = grd_ds.interp(height=np.arange(3150, 10050, 50), method='linear')
        
    # Resample the ground data to 5 min and interpolate to the CSU X-Band time. 
    # Keep data variable attributes to help distingish between instruments/locations
    if resample.split('=')[-1] == 'mean':
        matched = grd_ds.resample(time='5Min', 
                                  closed='right').mean(keep_attrs=True).interp(time=column.time, 
                                                                               method='linear')
    elif resample.split('=')[-1] == 'skip':
        matched = grd_ds.interp(time=column.time, method='linear')
    else:
        matched = grd_ds.resample(time='5Min', 
                                  closed='right').sum(keep_attrs=True).interp(time=column.time, 
                                                                              method='linear')
    
    # Add SAIL site location as a dimension for the Pluvio data
    matched = matched.assign_coords(coords=dict(site=site))
    matched = matched.expand_dims('site')
   
    # Remove Lat/Lon Data variables as it is included within the Matched Dataset with Site Identfiers
    if 'lat' in matched.data_vars:
        del matched['lat']
    if 'lon' in matched.data_vars:
        del matched['lon']
    if 'alt' in matched.data_vars:
        del matched['alt']
        
    # Update the individual Variables to Hold Global Attributes
    # global attributes will be lost on merging into the matched dataset.
    # Need to keep as many references and descriptors as possible
    for var in matched.data_vars:
        matched[var].attrs.update(source=matched.datastream)
        
    # Merge the two DataSets
    column = xr.merge([column, matched])
   
    return column

## Find / Download In-Situ Files

In [10]:
# define the datastream and output directory
insitu_stream = {'bnfmetM1.b1' : INSITU_DIR + 'bnfmetM1.b1',
                 'bnfmetS20.b1' : INSITU_DIR + "bnfmetS20.b1",
                 "bnfmetS30.b1" : INSITU_DIR + "bnfmetS30.b1",
                 "bnfmetS40.b1" : INSITU_DIR + "bnfmetS40.b1",
                 "bnfsondewnpnM1.b1" : INSITU_DIR + "bnfsondewnpnM1.b1",
                 "bnfwbpluvio2M1.a1" : INSITU_DIR + "bnfwbpluvio2M1.a1",
                 "bnfldquantsM1.c1" : INSITU_DIR + "bnfldquantsM1.c1",
                 "bnfldquantsS30.c1" : INSITU_DIR + "bnfldquantsS30.c1",
                 "bnfvdisquantsM1.c1" : INSITU_DIR + "bnfvdisquantsM1.c1"
                }

In [11]:
for var in insitu_stream:
    print(insitu_stream[var])

/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfmetM1.b1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfmetS20.b1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfmetS30.b1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfmetS40.b1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfsondewnpnM1.b1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfwbpluvio2M1.a1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfldquantsM1.c1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfldquantsS30.c1
/nfs/gce/globalscratch/obrienj/bnf-cmac-r2/in_situ/bnfvdisquantsM1.c1


In [14]:
# We can use the ACT module for downloading data from the ARM web service
for insitu in insitu_stream:
    #if insitu != "bnfsondewnpnM1.b1":
        results = act.discovery.download_arm_data(ARM_USERNAME, 
                                                  ARM_TOKEN, 
                                                  insitu, 
                                                  DATE, 
                                                  DATE, 
                                                  output=insitu_stream[insitu])

[DOWNLOADING] bnfmetM1.b1.20250109.000000.cdf

If you use these data to prepare a publication, please cite:

Kyrouac, J., Shi, Y., & Tuftedal, M. Surface Meteorological Instrumentation
(MET), 2025-01-09 to 2025-01-09, Bankhead National Forest, AL, USA; Long-term
Mobile Facility (BNF), Bankhead National Forest, AL, AMF3 (Main Site) (M1).
Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1786358

[DOWNLOADING] bnfmetS20.b1.20250109.000000.cdf

If you use these data to prepare a publication, please cite:

Kyrouac, J., Shi, Y., & Tuftedal, M. Surface Meteorological Instrumentation
(MET), 2025-01-09 to 2025-01-09, Bankhead National Forest, AL, USA; Long-term
Mobile Facility (BNF), Bankhead National Forest, AL, Supplemental facility at
Courtland (S20). Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1786358

[DOWNLOADING] bnfmetS30.b1.20250109.000000.cdf

If you use these data to prepare a publication, please cite:

Kyrouac, J., Shi

## Locate the CMAC Processed CSAPR-2 and Radiosonde Files

In [9]:
# With the user defined RADAR_DIR, grab all the XPRECIPRADAR CMAC files for the defined DATE
file_list = sorted(glob.glob(RADAR_DIR + 'bnfcsapr2cmacS3.c1.' + DATE.replace('-', '') + '*.nc'))
file_list[:10]

['/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.000253.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.000311.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.001255.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.001313.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.002258.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.002316.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.003301.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.003319.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.004303.nc',
 '/nfs/gce/globalscratch/obrienj/bnf-cmac/bnfcsapr2cmacS3.c1.20250305.004321.nc']

In [10]:
nsonde = sorted(glob.glob(insitu_stream["bnfsondewnpnM1.b1"] + '/*' + DATE.replace('-', '') + '*'))

In [17]:
%%time
## Start up a Dask Cluster
cluster = LocalCluster(n_workers=4)
with Client(cluster) as client:
    future = client.map(subset_points, file_list[:60], sonde=nsonde)
    my_data = client.gather(future)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2025-03-21 16:18:59,228 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 32.84 GiB -- Worker memory limit: 46.89 GiB
2025-03-21 16:19:17,333 - distributed.worker - ERROR - Compute Failed
Key:       subset_points-a8e4fafeec0306fe9eb1aa03addfe221
State:     long-running
Task:  <Task 'subset_points-a8e4fafeec0306fe9eb1aa03addfe221' subset_points(..., ...)>
Exception: 'ValueError("conflicting sizes for dimension \'station\': length 5 on \'station\' and length 1 on {\'station\': \'attenuation_corrected_differential_reflectivity\', \'height\': \'attenuation_corrected_differential_reflectivity\', \'time\': \'time\'}")'
Traceback: '  File "/tmp/ipykernel_1708717/427212614.py", line 115, in subset_points\n  File "/home/obrienj/miniforge3/envs/amf3-radar-example

ValueError: conflicting sizes for dimension 'station': length 5 on 'station' and length 1 on {'station': 'attenuation_corrected_differential_reflectivity', 'height': 'attenuation_corrected_differential_reflectivity', 'time': 'time'}

In [12]:
##%%time
##my_data = []
##j = 0
##for i in range(10, len(file_list), 10):
##    print(j, i)
##    iter_list = []
##    for scan in file_list[j:i]:
##        print(scan)
##        iter_list.append(subset_points(scan, sonde=nsonde))
##    print(len(iter_list))
##    if len(iter_list) > 0:
##        my_data.append(xr.concat([data for data in iter_list if data], dim='time'))
##    j += 10

## Combine all Extracted Radar Columns to Form Daily Timeseries

In [13]:
# Concatenate all extracted columns across time dimension to form daily timeseries
ds = xr.concat([data for data in my_data if data], dim='time')

NameError: name 'my_data' is not defined

In [18]:
cluster.close()

In [ ]:
ds

In [ ]:
ds = ds.drop_vars("base_time")

In [ ]:
ds.to_netcdf('bnf-radclss-test.nc')

## Read in the temporary saved file

In [ ]:
ds_rad = xr.open_dataset(file_list[10])

In [ ]:
ds_rad

In [ ]:
ds_rad.altitude.data

In [ ]:
ds_new = xr.open_dataset('bnf-radclss-test.nc')

In [ ]:
ds_new

In [ ]:
ds_new.height.data[-4]

In [ ]:
# Remove Global Attributes from the Column Extraction
# Attributes make sense for single location, but not collection of sites. 
ds.attrs = {}

In [ ]:
# Remove the Base_Time variable from extracted column
del ds['base_time']

In [ ]:
ds

## Define Dictionary of Variables to Remove from Each Datastream

In [ ]:
discard_var = {'LD' : ['base_time', 'time_offset', 'equivalent_radar_reflectivity_ott',
                       'laserband_amplitude', 'sensor_temperature', 'heating_current', 'sensor_voltage', 
                       'moment1', 'moment2', 'moment3', 'moment4',
                       'moment5', 'moment6', 'lat', 'lon', 'alt',
                       'qc_precip_rate', 'qc_weather_code', 'qc_equivalent_radar_reflectivity_ott',
                       'qc_number_detected_particles', 'qc_mor_visibility', 'qc_snow_depth_intensity',
                       'qc_laserband_amplitude', 'qc_heating_current', 'qc_sensor_voltage'
                      ],
               'Pluvio' : ['base_time', 'time_offset', 'load_cell_temp', 'heater_status',
                          'elec_unit_temp', 'supply_volts', 'orifice_temp', 'volt_min',
                          'ptemp', 'lat', 'lon', 'alt', 'maintenance_flag', 'reset_flag', 
                          'qc_rh_mean', 'pluvio_status'
                          ],
               'Met' : ['base_time', 'time_offset', 'time_bounds', 'logger_volt',
                        'logger_temp', 'qc_logger_temp', 'lat', 'lon', 'alt', 'qc_temp_mean',
                        'qc_rh_mean', 'qc_vapor_pressure_mean', 'qc_wspd_arith_mean', 'qc_wspd_vec_mean',
                        'qc_wdir_vec_mean', 'qc_pwd_mean_vis_1min', 'qc_pwd_mean_vis_10min', 'qc_pwd_pw_code_inst',
                        'qc_pwd_pw_code_15min', 'qc_pwd_pw_code_1hr', 'qc_pwd_precip_rate_mean_1min',
                        'qc_pwd_cumul_rain', 'qc_pwd_cumul_snow', 'qc_org_precip_rate_mean', 'qc_tbrg_precip_total',
                        'qc_tbrg_precip_total_corr', 'qc_logger_volt', 'qc_logger_temp', 'qc_atmos_pressure', 
                        'pwd_pw_code_inst', 'pwd_pw_code_15min', 'pwd_pw_code_1hr', 
                       ],
               'RWP' : ['base_time', 'time_offset', 'time_bounds', 'height_bounds', 'vertical_wind_speed_count',
                        'vertical_wind_speed_quality_flag', 'lat', 'lon'
                       ],
               'ceil': ['base_time', 'time_offset', 'time_bounds', 'range_bounds', 'detection_status',
                        'status_flag', 'qc_first_cbh', 'qc_vertical_visibility', 'qc_second_cbh', 'qc_alt_highest_signal', 
                        'qc_third_cbh', 'laser_pulse_energy', 'qc_laser_pulse_energy', 'laser_temperature', 'qc_laser_temperature',
                        'window_transmission', 'qc_window_transmission', 'tilt_angle', 'qc_tilt_angle', 'background_light',
                        'qc_background_light', 'sum_backscatter', 'qc_sum_backscatter', 'measurement_parameters', 'status_string',
                        'alt_highest_signal', 'lat', 'lon'
                       ]
              }

## Save the DataSet

In [ ]:
# define a filename
out_ds.to_netcdf('xprecipradarradclss.c2.' + DATE.replace('-', '') + '.000000.nc')

## References

### Surface Meteorological Instrumentation (MET)
- Kyrouac, J., Shi, Y., & Tuftedal, M. Surface Meteorological Instrumentation
(MET), 2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term
Mobile Facility (BNF), Bankhead National Forest, AL, AMF3 (Main Site) **(M1)**.
Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1786358

- Kyrouac, J., Shi, Y., & Tuftedal, M. Surface Meteorological Instrumentation
(MET), 2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term
Mobile Facility (BNF), Bankhead National Forest, AL, Supplemental facility at
Courtland **(S20)**. Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1786358

- Kyrouac, J., Shi, Y., & Tuftedal, M. Surface Meteorological Instrumentation
(MET), 2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term
Mobile Facility (BNF), Bankhead National Forest, AL, Supplemental facility at
Falkville **(S30)**. Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1786358

- Kyrouac, J., Shi, Y., & Tuftedal, M. Surface Meteorological Instrumentation
(MET), 2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term
Mobile Facility (BNF), Bankhead National Forest, AL, Supplemental facility at
Double Springs **(S40)**. Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1786358

### Balloon-Borne Sounding System (SONDEWNPN)
- Keeler, E., Burk, K., & Kyrouac, J. Balloon-Borne Sounding System (SONDEWNPN),
2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term Mobile
Facility (BNF), Bankhead National Forest, AL, AMF3 (Main Site) **(M1)**. Atmospheric
Radiation Measurement (ARM) User Facility. https://doi.org/10.5439/1595321

### Weighing Bucket Preciptiation Gauge (WBPLUVIO2)
- Zhu, Z., Wang, D., Jane, M., Cromwell, E., Sturm, M., Irving, K., & Delamere, J.
Weighing Bucket Precipitation Gauge (WBPLUVIO2), 2025-03-05 to 2025-03-05,
Bankhead National Forest, AL, USA; Long-term Mobile Facility (BNF), Bankhead
National Forest, AL, AMF3 (Main Site) **(M1)**. Atmospheric Radiation Measurement
(ARM) User Facility. https://doi.org/10.5439/1338194

### Laser Disdrometer Quantities (LDQUANTS)
- Hardin, J., Giangrande, S., & Zhou, A. Laser Disdrometer Quantities (LDQUANTS),
2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term Mobile
Facility (BNF), Bankhead National Forest, AL, AMF3 (Main Site) **(M1)**. Atmospheric
Radiation Measurement (ARM) User Facility. https://doi.org/10.5439/1432694

- Hardin, J., Giangrande, S., & Zhou, A. Laser Disdrometer Quantities (LDQUANTS),
2025-03-05 to 2025-03-05, Bankhead National Forest, AL, USA; Long-term Mobile
Facility (BNF), Bankhead National Forest, AL, Supplemental facility at Falkville
**(S30)**. Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1432694

### Video Disdrometer Quantities (VDISQUANTS)
